# 📖 VerbiAI - CrewAI Dictionary & Web Search Agent
This notebook defines **VerbiAI**, a **CrewAI-based linguistic agent** that:  
- **Retrieves word definitions, synonyms, and antonyms**  
- **Performs a Google Search for contextual understanding**  
- **Generates a short summary of search results**  

## 🛠 Features:
- **CrewAI-powered** agent management  
- **Uses NLTK's WordNet for linguistic data**  
- **Fetches relevant web results for enhanced word insights**  

🔍 **Run the cells below to initialize VerbiAI!**


In [2]:
# Import necessary libraries
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool
import requests
import re
import nltk
from nltk.corpus import wordnet
from bs4 import BeautifulSoup

# Ensure NLTK WordNet is downloaded
nltk.download('wordnet')


e:\AIAgents\agents\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
e:\AIAgents\agents\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
e:\AIAgents\agents\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration

True

In [3]:
# Web scraping tool for Google image search
google_scraper = ScrapeWebsiteTool(website_url="https://www.google.com/")

In [5]:
# Query Classification Function
def classify_query(user_query):
    """
    Determines if a query is valid for dictionary lookup.
    """
    user_query = user_query.lower().strip()
    
    # Acceptable word query format (only alphabets, no numbers/special characters)
    if re.fullmatch(r"[a-zA-Z\s]+", user_query):
        print(f"DEBUG: Query '{user_query}' classified as 'dictionary_lookup'")
        return "dictionary_lookup"

    print(f"DEBUG: Query '{user_query}' classified as 'unknown'")
    return "unknown"


In [6]:
# Query Classifier Agent
query_classifier = Agent(
    role="Dictionary Query Classifier",
    goal="Classify queries and direct them to the Dictionary Lookup Agent.",
    backstory="You determine if a user's query is a valid word for dictionary lookup.",
    allow_delegation=True,
    verbose=True
)

In [7]:
# Dictionary Lookup Agent
class DictionaryLookupAgent(Agent):
    def execute(self, word):
        """
        Fetches definition, synonyms, and example usage from WordNet.
        """
        try:
            synsets = wordnet.synsets(word)
            if not synsets:
                return f"Sorry, no information found for '{word}'."

            definition = synsets[0].definition()
            synonyms = list(set(lemma.name().replace("_", " ") for synset in synsets for lemma in synset.lemmas()))[:5]
            example = synsets[0].examples()[0] if synsets[0].examples() else "No example available."

            return {
                "word": word.capitalize(),
                "definition": definition,
                "synonyms": synonyms,
                "example": example
            }
        except Exception as e:
            return f"Error fetching word details: {str(e)}"

In [8]:
dictionary_agent = DictionaryLookupAgent(
    role="Dictionary Lookup Agent",
    goal="Retrieve the definition, synonyms, and example usage of a given word.",
    backstory="You are a linguistic expert providing dictionary definitions and word details.",
    allow_delegation=True,
    verbose=True
)

In [9]:
# Image Search Agent
class ImageSearchAgent(Agent):
    def execute(self, query):
        """
        Performs Google image search and extracts image links.
        """
        try:
            search_url = f"https://www.google.com/search?tbm=isch&q={query.replace(' ', '+')}"
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(search_url, headers=headers)

            if response.status_code != 200:
                return []

            soup = BeautifulSoup(response.text, "html.parser")
            image_tags = soup.find_all("img")

            # Extract first 3 image URLs (ignoring the first, usually a Google logo)
            image_urls = [img["src"] for img in image_tags[1:4] if "http" in img["src"]]
            return image_urls
        except Exception as e:
            return f"Error fetching images: {str(e)}"

In [10]:
image_search_agent = ImageSearchAgent(
    role="Image Search Agent",
    goal="Find relevant images for a given word using Google search.",
    backstory="You are a web search expert retrieving top related images for words.",
    tools=[google_scraper],
    allow_delegation=True,
    verbose=True
)

In [11]:
# Tasks for Each Agent
dictionary_task = Task(
    description="Fetch the definition, synonyms, and example usage for the word {query}.",
    expected_output="A dictionary-style response with a definition, synonyms, and an example.",
    agent=dictionary_agent
)

In [12]:

image_task = Task(
    description="Perform a Google image search for {query} and retrieve 3 best image links.",
    expected_output="A list of URLs pointing to relevant images.",
    agent=image_search_agent
)


In [13]:
# CrewAI Orchestration
class VerbiAICrew:
    def __init__(self):
        self.crew = Crew(
            agents=[query_classifier, dictionary_agent, image_search_agent],
            tasks=[dictionary_task, image_task],
            verbose=True,
            memory=False
        )

    def run(self, word):
        category = classify_query(word)
        if category != "dictionary_lookup":
            return "Sorry, I couldn't understand your query."

        print("DEBUG: Sending query to Dictionary Agent")
        word_info = dictionary_agent.execute(word)

        print("DEBUG: Sending query to Image Search Agent")
        image_links = image_search_agent.execute(word)

        # Format the response
        response = f"📖 **Word:** {word_info['word']}\n"
        response += f"📝 **Definition:** {word_info['definition']}\n"
        response += f"🔄 **Synonyms:** {', '.join(word_info['synonyms'])}\n"
        response += f"📌 **Example Usage:** \"{word_info['example']}\"\n\n"

        if image_links:
            response += "🖼 **Image Links:**\n" + "\n".join([f"{i+1}️⃣ {link}" for i, link in enumerate(image_links)])
        else:
            response += "🚫 No images found."

        return response

In [ ]:
# ✅ Single Query Execution
verbi_ai_crew = VerbiAICrew()
user_query = input("Enter a word to define: Happy ")
print(f"Query: {user_query} ➝ Result: \n{verbi_ai_crew.run(user_query)}")